In [1]:
import  asyncio, nest_asyncio
from typing import List
import pandas as pd

from read_telega_dump import telega_dump_to_pandas
from read_telega_api import extract_message_reactions
import time

nest_asyncio.apply()

dump_path = r"D:\test_data\ChatExport_2024-03-14\result.json"
tlg_group_id = -1001688539638 
# telegram group id, can be taken from "copy message link"  in desctop telegram

In [2]:
async def jpt_ntb_extract_message_reactions(tlg_group_id: int, ids: List[int])->pd.DataFrame:
    loop = asyncio.get_event_loop()
    task = loop.create_task(extract_message_reactions(tlg_group_id, ids, chunk_size=50000))
    result = await task
    # print("Result:", result)
    return result

In [3]:
df = telega_dump_to_pandas(dump_path=dump_path)


In [4]:
ids = df['id'].to_list()
df_reactions = asyncio.run(jpt_ntb_extract_message_reactions(tlg_group_id, ids))



INFO:root:need to process 149189 messages
INFO:telethon.network.mtprotosender:Connecting to 149.154.167.51:443/TcpFull...
INFO:telethon.network.mtprotosender:Connection to 149.154.167.51:443/TcpFull complete!
INFO:root:retrieving reactions for 50000 messages
INFO:root:got reactions for 9 messages at 21:05:53.095643
INFO:root:retrieving reactions for 50000 messages
INFO:root:got reactions for 30 messages at 21:05:55.587720
INFO:root:retrieving reactions for 49189 messages
INFO:root:got reactions for 52 messages at 21:05:58.171594
INFO:telethon.network.mtprotosender:Disconnecting from 149.154.167.51:443/TcpFull...
INFO:telethon.network.mtprotosender:Disconnection from 149.154.167.51:443/TcpFull complete!


AttributeError: 'ReactionCustomEmoji' object has no attribute 'emoticon'

In [ ]:
df.to_parquet('df.parquet.gzip', compression='gzip')

In [ ]:
import tiktoken

# Load the cl100k_base tokenizer which is designed to work with the ada-002 model
tokenizer = tiktoken.get_encoding("cl100k_base")


# Tokenize the text and save the number of tokens to a new column
df['n_tokens'] = df.mess_text.apply(lambda x: len(tokenizer.encode(x)))



In [ ]:
# df.dtypes
# Visualize the distribution of the number of tokens per row using a histogram
df.n_tokens.hist(bins=200)
# df.tail(50)

